In [1]:
"""
IMPORTS
"""

import pandas as pd
import numpy as np
import onnxruntime as rt
import dice_ml
from dice_ml.utils import helpers 
import shap  # package used to calculate Shap values
\
from utils.dataloader import DataLoader
from models.modelloader import ModelLoader
from utils.constants import protected_attributes, group_proxies


from tests.metrics import MetricsTester
from tests.mututation import MutationTester
from tests.metamorphic import MetamorphicTester
from tests.shapley import ShapleyTester

Initializing test: data_shuffler...
Initializing test: data_remover...
Initializing test: data_repetition...
Initializing test: label_error...
Initializing test: feature_remover...


In [2]:
"""
    INTERESTING LINKS:
        - https://dl.acm.org/doi/pdf/10.1145/3338906.3338937
"""

'\n    INTERESTING LINKS:\n        - https://dl.acm.org/doi/pdf/10.1145/3338906.3338937\n'

In [3]:
"""
SET MODEL(S)
"""

params = {
    'learning_rate': 0.15
}

model_loader = ModelLoader(type='good', params=params)

In [4]:
"""
LOAD DATASET
"""

dataloader = DataLoader()
X_train, y_train = dataloader.load_split('train')
print(f'{X_train.shape}, {y_train.shape}')
X_test, y_test = dataloader.load_split('test')
print(f'{X_test.shape}, {y_test.shape}')
X_data, y_data = dataloader.load_split('full')
print(f'{X_data.shape}, {y_data.shape}')
dataset_train_test = pd.concat([X_data, y_data], axis=1)
print(f'{dataset_train_test.shape}')


(10116, 315), (10116,)
(2529, 315), (2529,)
(12645, 315), (12645,)
(12645, 316)


In [5]:
"""
    TEST MODEL ON DATASET
"""

model_loader.fit(X_train, y_train)


In [6]:
y_pred = model_loader.predict(X_test)
print(y_pred)

[0 0 0 ... 0 0 0]


In [7]:
"""
FUNCTIONAL TESTING
"""

metrics = MetricsTester(protected_variables=protected_attributes)


In [8]:
print(X_test.shape)
print(y_test.shape)
print(y_pred.shape)
metrics.get_metrics_summary(X_test, y_true=y_test, y_pred=y_pred)

(2529, 315)
(2529,)
(2529,)
Overal metrics: 
TN-FP-FN-TP    (2240, 20, 147, 122)
acc                        0.933966
prec                       0.859155
rec                        0.453532
f1                         0.593674
fnr                        0.546468
fpr                         0.00885
sel                        0.056149
count                          2529
dtype: object 
Metrics for group: 
                             fnr       fpr       sel  count
adres_aantal_brp_adres                                     
1                       0.718750  0.008043  0.029630  405.0
2                       0.552239  0.008380  0.045977  783.0
3                       0.468750  0.011429  0.067912  589.0
4                       0.500000  0.005882  0.064767  386.0
5                       0.611111  0.005208  0.065789  228.0
6                       0.562500  0.023810  0.090000  100.0
7                       0.200000  0.000000  0.137931   29.0
8                       1.000000  0.000000  0.000000    

In [9]:
"""
MUTATION TESTING
"""

params = {
    'data_shuffler': 1000,
    'data_remover': 0.25,
    'data_repetition': 1000, 
    'label_error': 1000,
    'feature_remover': 31,
}

mutation  = False
if mutation==True:
    mutator = MutationTester(y_pred_baseline=y_pred, df_train=X_train, df_test=X_test)
    mutation_score = mutator.test_mutants(params)
    print(f"Mutation score: {mutation_score}")

In [10]:
"""
AUC
"""


'\nAUC\n'

In [22]:
"""
SHAP VALUES: see kaggle notebook
"""

'\nSHAP VALUES: see kaggle notebook\n'

In [23]:
"""
COUNTERFACTUALS
"""

# Model

data = dice_ml.Data(dataframe=dataset_train_test, 
                   continuous_features=["persoon_leeftijd_bij_onderzoek"],
                   outcome_name="checked")

rf_dice = dice_ml.Model(model=model_loader.model.model, 
                        # There exist backends for tf, torch, ...
                        backend="sklearn")

explainer = dice_ml.Dice(data, 
                         rf_dice, 
                         # Random sampling, genetic algorithm, kd-tree,...
                         method="random")


In [24]:
original_input = X_test[0:4]
cf_examples = explainer.generate_counterfactuals(original_input,
                                           total_CFs=3,
                                           desired_class="opposite")
cf_examples.visualize_as_dataframe(show_only_changes=True)

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:11<00:00,  2.78s/it]

Query instance (original outcome : 0)


,adres_aantal_brp_adres,adres_aantal_verschillende_wijken,adres_aantal_verzendadres,adres_aantal_woonadres_handmatig,adres_dagen_op_adres,adres_recentst_onderdeel_rdam,adres_recentste_buurt_groot_ijsselmonde,adres_recentste_buurt_nieuwe_westen,adres_recentste_buurt_other,adres_recentste_buurt_oude_noorden,adres_recentste_buurt_vreewijk,adres_recentste_plaats_other,adres_recentste_plaats_rotterdam,adres_recentste_wijk_charlois,adres_recentste_wijk_delfshaven,adres_recentste_wijk_feijenoord,adres_recentste_wijk_ijsselmonde,adres_recentste_wijk_kralingen_c,adres_recentste_wijk_noord,adres_recentste_wijk_other,adres_recentste_wijk_prins_alexa,adres_recentste_wijk_stadscentru,adres_unieke_wijk_ratio,afspraak_aanmelding_afgesloten,afspraak_aantal_woorden,afspraak_afgelopen_jaar_afsprakenplan,afspraak_afgelopen_jaar_monitoring_insp__wet_taaleis_na_12_mnd_n_a_v__taa04_____geen_maatregel,afspraak_afgelopen_jaar_ontheffing,afspraak_afgelopen_jaar_ontheffing_taaleis,afspraak_afgelopen_jaar_plan_van_aanpak,afspraak_afgelopen_jaar_signaal_voor_medewerker,afspraak_afgelopen_jaar_vervolgmeting_matchbaarheid_werkzoekende_klant,afspraak_afgelopen_jaar_voortgang_aanmelding_en_deelname,afspraak_afsprakenplan,afspraak_controle_aankondiging_maatregel,afspraak_controle_verwijzing,afspraak_deelname_compleet_uit_webapplicatie,afspraak_galo_gesprek,afspraak_gespr__einde_zoekt___galo_gesprek_,afspraak_inspanningsperiode,afspraak_laatstejaar_aantal_woorden,afspraak_laatstejaar_resultaat_ingevuld,afspraak_laatstejaar_resultaat_ingevuld_uniek,afspraak_other,afspraak_participatietrede_vervolgmeting,afspraak_resultaat_ingevuld_uniek,afspraak_signaal_van_aanbieder,afspraak_signaal_voor_medewerker,afspraak_toevoegen_inschrijving_uwvwb,afspraak_vervolgmeting_matchbaarheid_werkzoekende_klant,afspraak_verzenden_beschikking_i_v_m__niet_voldoen_aan_wet_taaleis,afspraak_voortgangsgesprek,belemmering_aantal_huidig,belemmering_dagen_financiele_problemen,belemmering_dagen_lichamelijke_problematiek,belemmering_dagen_psychische_problemen,belemmering_financiele_problemen,belemmering_hist_lichamelijke_problematiek,belemmering_hist_psychische_problemen,belemmering_hist_stabiele_mix__sz____dagbesteding_werk,belemmering_hist_taal,belemmering_hist_verslavingsproblematiek,belemmering_ind,belemmering_ind_hist,belemmering_niet_computervaardig,belemmering_psychische_problemen,belemmering_woonsituatie,beschikbaarheid_aantal_historie_afwijkend_wegens_medische_omstandigheden,beschikbaarheid_aantal_historie_afwijkend_wegens_sociaal_maatschappelijke_situatie,beschikbaarheid_huidig_afwijkend_wegens_medische_omstandigheden,beschikbaarheid_huidig_bekend,beschikbaarheid_recent_afwijkend_wegens_medische_omstandigheden,beschikbaarheid_recent_afwijkend_wegens_sociaal_maatschappelijke_situatie,competentie_aansturen,competentie_analyseren,competentie_ethisch_en_integer_handelen,competentie_formuleren_en_rapporteren,competentie_gedrevenheid_en_ambitie_tonen,competentie_instructies_en_procedures_opvolgen,competentie_kwaliteit_leveren,competentie_leren,competentie_materialen_en_middelen_inzetten,competentie_met_druk_en_tegenslag_omgaan,competentie_omgaan_met_verandering_en_aanpassen,competentie_onderzoeken,competentie_op_de_behoeften_en_verwachtingen_van_de__klant__richten,competentie_other,competentie_overtuigen_en_beïnvloeden,competentie_plannen_en_organiseren,competentie_samenwerken_en_overleggen,competentie_vakdeskundigheid_toepassen,contacten_onderwerp__arbeids_motivatie,contacten_onderwerp__pre__intake,contacten_onderwerp__werk_intake,contacten_onderwerp_arbeidsdiagnose_dariuz,contacten_onderwerp_beoordelen_taaleis,contacten_onderwerp_boolean__arbeids_motivatie,contacten_onderwerp_boolean__pre__intake,contacten_onderwerp_boolean__werk_intake,contacten_onderwerp_boolean_beoordelen_taaleis,contacten_onderwerp_boolean_contact_derden,contacten_onderwerp_boolean_contact_met_aanbieder,contacten_onderwerp_boolean_diagnosegesprek,contacten_onderwerp_boolean_documenten__innemen_,contacten_onderwerp_


Diverse Counterfactual set (new outcome: 1)


...

,adres_aantal_brp_adres,adres_aantal_verschillende_wijken,adres_aantal_verzendadres,adres_aantal_woonadres_handmatig,adres_dagen_op_adres,adres_recentst_onderdeel_rdam,adres_recentste_buurt_groot_ijsselmonde,adres_recentste_buurt_nieuwe_westen,adres_recentste_buurt_other,adres_recentste_buurt_oude_noorden,adres_recentste_buurt_vreewijk,adres_recentste_plaats_other,adres_recentste_plaats_rotterdam,adres_recentste_wijk_charlois,adres_recentste_wijk_delfshaven,adres_recentste_wijk_feijenoord,adres_recentste_wijk_ijsselmonde,adres_recentste_wijk_kralingen_c,adres_recentste_wijk_noord,adres_recentste_wijk_other,adres_recentste_wijk_prins_alexa,adres_recentste_wijk_stadscentru,adres_unieke_wijk_ratio,afspraak_aanmelding_afgesloten,afspraak_aantal_woorden,afspraak_afgelopen_jaar_afsprakenplan,afspraak_afgelopen_jaar_monitoring_insp__wet_taaleis_na_12_mnd_n_a_v__taa04_____geen_maatregel,afspraak_afgelopen_jaar_ontheffing,afspraak_afgelopen_jaar_ontheffing_taaleis,afspraak_afgelopen_jaar_plan_van_aanpak,afspraak_afgelopen_jaar_signaal_voor_medewerker,afspraak_afgelopen_jaar_vervolgmeting_matchbaarheid_werkzoekende_klant,afspraak_afgelopen_jaar_voortgang_aanmelding_en_deelname,afspraak_afsprakenplan,afspraak_controle_aankondiging_maatregel,afspraak_controle_verwijzing,afspraak_deelname_compleet_uit_webapplicatie,afspraak_galo_gesprek,afspraak_gespr__einde_zoekt___galo_gesprek_,afspraak_inspanningsperiode,afspraak_laatstejaar_aantal_woorden,afspraak_laatstejaar_resultaat_ingevuld,afspraak_laatstejaar_resultaat_ingevuld_uniek,afspraak_other,afspraak_participatietrede_vervolgmeting,afspraak_resultaat_ingevuld_uniek,afspraak_signaal_van_aanbieder,afspraak_signaal_voor_medewerker,afspraak_toevoegen_inschrijving_uwvwb,afspraak_vervolgmeting_matchbaarheid_werkzoekende_klant,afspraak_verzenden_beschikking_i_v_m__niet_voldoen_aan_wet_taaleis,afspraak_voortgangsgesprek,belemmering_aantal_huidig,belemmering_dagen_financiele_problemen,belemmering_dagen_lichamelijke_problematiek,belemmering_dagen_psychische_problemen,belemmering_financiele_problemen,belemmering_hist_lichamelijke_problematiek,belemmering_hist_psychische_problemen,belemmering_hist_stabiele_mix__sz____dagbesteding_werk,belemmering_hist_taal,belemmering_hist_verslavingsproblematiek,belemmering_ind,belemmering_ind_hist,belemmering_niet_computervaardig,belemmering_psychische_problemen,belemmering_woonsituatie,beschikbaarheid_aantal_historie_afwijkend_wegens_medische_omstandigheden,beschikbaarheid_aantal_historie_afwijkend_wegens_sociaal_maatschappelijke_situatie,beschikbaarheid_huidig_afwijkend_wegens_medische_omstandigheden,beschikbaarheid_huidig_bekend,beschikbaarheid_recent_afwijkend_wegens_medische_omstandigheden,beschikbaarheid_recent_afwijkend_wegens_sociaal_maatschappelijke_situatie,competentie_aansturen,competentie_analyseren,competentie_ethisch_en_integer_handelen,competentie_formuleren_en_rapporteren,competentie_gedrevenheid_en_ambitie_tonen,competentie_instructies_en_procedures_opvolgen,competentie_kwaliteit_leveren,competentie_leren,competentie_materialen_en_middelen_inzetten,competentie_met_druk_en_tegenslag_omgaan,competentie_omgaan_met_verandering_en_aanpassen,competentie_onderzoeken,competentie_op_de_behoeften_en_verwachtingen_van_de__klant__richten,competentie_other,competentie_overtuigen_en_beïnvloeden,competentie_plannen_en_organiseren,competentie_samenwerken_en_overleggen,competentie_vakdeskundigheid_toepassen,contacten_onderwerp__arbeids_motivatie,contacten_onderwerp__pre__intake,contacten_onderwerp__werk_intake,contacten_onderwerp_arbeidsdiagnose_dariuz,contacten_onderwerp_beoordelen_taaleis,contacten_onderwerp_boolean__arbeids_motivatie,contacten_onderwerp_boolean__pre__intake,contacten_onderwerp_boolean__werk_intake,contacten_onderwerp_boolean_beoordelen_taaleis,contacten_onderwerp_boolean_contact_derden,contacten_onderwerp_boolean_contact_met_aanbieder,contacten_onderwerp_boolean_diagnosegesprek,contacten_onderwerp_boolean_documenten__innemen_,contacten_onderwerp_

Query instance (original outcome : 0)


,adres_aantal_brp_adres,adres_aantal_verschillende_wijken,adres_aantal_verzendadres,adres_aantal_woonadres_handmatig,adres_dagen_op_adres,adres_recentst_onderdeel_rdam,adres_recentste_buurt_groot_ijsselmonde,adres_recentste_buurt_nieuwe_westen,adres_recentste_buurt_other,adres_recentste_buurt_oude_noorden,adres_recentste_buurt_vreewijk,adres_recentste_plaats_other,adres_recentste_plaats_rotterdam,adres_recentste_wijk_charlois,adres_recentste_wijk_delfshaven,adres_recentste_wijk_feijenoord,adres_recentste_wijk_ijsselmonde,adres_recentste_wijk_kralingen_c,adres_recentste_wijk_noord,adres_recentste_wijk_other,adres_recentste_wijk_prins_alexa,adres_recentste_wijk_stadscentru,adres_unieke_wijk_ratio,afspraak_aanmelding_afgesloten,afspraak_aantal_woorden,afspraak_afgelopen_jaar_afsprakenplan,afspraak_afgelopen_jaar_monitoring_insp__wet_taaleis_na_12_mnd_n_a_v__taa04_____geen_maatregel,afspraak_afgelopen_jaar_ontheffing,afspraak_afgelopen_jaar_ontheffing_taaleis,afspraak_afgelopen_jaar_plan_van_aanpak,afspraak_afgelopen_jaar_signaal_voor_medewerker,afspraak_afgelopen_jaar_vervolgmeting_matchbaarheid_werkzoekende_klant,afspraak_afgelopen_jaar_voortgang_aanmelding_en_deelname,afspraak_afsprakenplan,afspraak_controle_aankondiging_maatregel,afspraak_controle_verwijzing,afspraak_deelname_compleet_uit_webapplicatie,afspraak_galo_gesprek,afspraak_gespr__einde_zoekt___galo_gesprek_,afspraak_inspanningsperiode,afspraak_laatstejaar_aantal_woorden,afspraak_laatstejaar_resultaat_ingevuld,afspraak_laatstejaar_resultaat_ingevuld_uniek,afspraak_other,afspraak_participatietrede_vervolgmeting,afspraak_resultaat_ingevuld_uniek,afspraak_signaal_van_aanbieder,afspraak_signaal_voor_medewerker,afspraak_toevoegen_inschrijving_uwvwb,afspraak_vervolgmeting_matchbaarheid_werkzoekende_klant,afspraak_verzenden_beschikking_i_v_m__niet_voldoen_aan_wet_taaleis,afspraak_voortgangsgesprek,belemmering_aantal_huidig,belemmering_dagen_financiele_problemen,belemmering_dagen_lichamelijke_problematiek,belemmering_dagen_psychische_problemen,belemmering_financiele_problemen,belemmering_hist_lichamelijke_problematiek,belemmering_hist_psychische_problemen,belemmering_hist_stabiele_mix__sz____dagbesteding_werk,belemmering_hist_taal,belemmering_hist_verslavingsproblematiek,belemmering_ind,belemmering_ind_hist,belemmering_niet_computervaardig,belemmering_psychische_problemen,belemmering_woonsituatie,beschikbaarheid_aantal_historie_afwijkend_wegens_medische_omstandigheden,beschikbaarheid_aantal_historie_afwijkend_wegens_sociaal_maatschappelijke_situatie,beschikbaarheid_huidig_afwijkend_wegens_medische_omstandigheden,beschikbaarheid_huidig_bekend,beschikbaarheid_recent_afwijkend_wegens_medische_omstandigheden,beschikbaarheid_recent_afwijkend_wegens_sociaal_maatschappelijke_situatie,competentie_aansturen,competentie_analyseren,competentie_ethisch_en_integer_handelen,competentie_formuleren_en_rapporteren,competentie_gedrevenheid_en_ambitie_tonen,competentie_instructies_en_procedures_opvolgen,competentie_kwaliteit_leveren,competentie_leren,competentie_materialen_en_middelen_inzetten,competentie_met_druk_en_tegenslag_omgaan,competentie_omgaan_met_verandering_en_aanpassen,competentie_onderzoeken,competentie_op_de_behoeften_en_verwachtingen_van_de__klant__richten,competentie_other,competentie_overtuigen_en_beïnvloeden,competentie_plannen_en_organiseren,competentie_samenwerken_en_overleggen,competentie_vakdeskundigheid_toepassen,contacten_onderwerp__arbeids_motivatie,contacten_onderwerp__pre__intake,contacten_onderwerp__werk_intake,contacten_onderwerp_arbeidsdiagnose_dariuz,contacten_onderwerp_beoordelen_taaleis,contacten_onderwerp_boolean__arbeids_motivatie,contacten_onderwerp_boolean__pre__intake,contacten_onderwerp_boolean__werk_intake,contacten_onderwerp_boolean_beoordelen_taaleis,contacten_onderwerp_boolean_contact_derden,contacten_onderwerp_boolean_contact_met_aanbieder,contacten_onderwerp_boolean_diagnosegesprek,contacten_onderwerp_boolean_documenten__innemen_,contacten_onderwerp_


Diverse Counterfactual set (new outcome: 1)


,adres_aantal_brp_adres,adres_aantal_verschillende_wijken,adres_aantal_verzendadres,adres_aantal_woonadres_handmatig,adres_dagen_op_adres,adres_recentst_onderdeel_rdam,adres_recentste_buurt_groot_ijsselmonde,adres_recentste_buurt_nieuwe_westen,adres_recentste_buurt_other,adres_recentste_buurt_oude_noorden,adres_recentste_buurt_vreewijk,adres_recentste_plaats_other,adres_recentste_plaats_rotterdam,adres_recentste_wijk_charlois,adres_recentste_wijk_delfshaven,adres_recentste_wijk_feijenoord,adres_recentste_wijk_ijsselmonde,adres_recentste_wijk_kralingen_c,adres_recentste_wijk_noord,adres_recentste_wijk_other,adres_recentste_wijk_prins_alexa,adres_recentste_wijk_stadscentru,adres_unieke_wijk_ratio,afspraak_aanmelding_afgesloten,afspraak_aantal_woorden,afspraak_afgelopen_jaar_afsprakenplan,afspraak_afgelopen_jaar_monitoring_insp__wet_taaleis_na_12_mnd_n_a_v__taa04_____geen_maatregel,afspraak_afgelopen_jaar_ontheffing,afspraak_afgelopen_jaar_ontheffing_taaleis,afspraak_afgelopen_jaar_plan_van_aanpak,afspraak_afgelopen_jaar_signaal_voor_medewerker,afspraak_afgelopen_jaar_vervolgmeting_matchbaarheid_werkzoekende_klant,afspraak_afgelopen_jaar_voortgang_aanmelding_en_deelname,afspraak_afsprakenplan,afspraak_controle_aankondiging_maatregel,afspraak_controle_verwijzing,afspraak_deelname_compleet_uit_webapplicatie,afspraak_galo_gesprek,afspraak_gespr__einde_zoekt___galo_gesprek_,afspraak_inspanningsperiode,afspraak_laatstejaar_aantal_woorden,afspraak_laatstejaar_resultaat_ingevuld,afspraak_laatstejaar_resultaat_ingevuld_uniek,afspraak_other,afspraak_participatietrede_vervolgmeting,afspraak_resultaat_ingevuld_uniek,afspraak_signaal_van_aanbieder,afspraak_signaal_voor_medewerker,afspraak_toevoegen_inschrijving_uwvwb,afspraak_vervolgmeting_matchbaarheid_werkzoekende_klant,afspraak_verzenden_beschikking_i_v_m__niet_voldoen_aan_wet_taaleis,afspraak_voortgangsgesprek,belemmering_aantal_huidig,belemmering_dagen_financiele_problemen,belemmering_dagen_lichamelijke_problematiek,belemmering_dagen_psychische_problemen,belemmering_financiele_problemen,belemmering_hist_lichamelijke_problematiek,belemmering_hist_psychische_problemen,belemmering_hist_stabiele_mix__sz____dagbesteding_werk,belemmering_hist_taal,belemmering_hist_verslavingsproblematiek,belemmering_ind,belemmering_ind_hist,belemmering_niet_computervaardig,belemmering_psychische_problemen,belemmering_woonsituatie,beschikbaarheid_aantal_historie_afwijkend_wegens_medische_omstandigheden,beschikbaarheid_aantal_historie_afwijkend_wegens_sociaal_maatschappelijke_situatie,beschikbaarheid_huidig_afwijkend_wegens_medische_omstandigheden,beschikbaarheid_huidig_bekend,beschikbaarheid_recent_afwijkend_wegens_medische_omstandigheden,beschikbaarheid_recent_afwijkend_wegens_sociaal_maatschappelijke_situatie,competentie_aansturen,competentie_analyseren,competentie_ethisch_en_integer_handelen,competentie_formuleren_en_rapporteren,competentie_gedrevenheid_en_ambitie_tonen,competentie_instructies_en_procedures_opvolgen,competentie_kwaliteit_leveren,competentie_leren,competentie_materialen_en_middelen_inzetten,competentie_met_druk_en_tegenslag_omgaan,competentie_omgaan_met_verandering_en_aanpassen,competentie_onderzoeken,competentie_op_de_behoeften_en_verwachtingen_van_de__klant__richten,competentie_other,competentie_overtuigen_en_beïnvloeden,competentie_plannen_en_organiseren,competentie_samenwerken_en_overleggen,competentie_vakdeskundigheid_toepassen,contacten_onderwerp__arbeids_motivatie,contacten_onderwerp__pre__intake,contacten_onderwerp__werk_intake,contacten_onderwerp_arbeidsdiagnose_dariuz,contacten_onderwerp_beoordelen_taaleis,contacten_onderwerp_boolean__arbeids_motivatie,contacten_onderwerp_boolean__pre__intake,contacten_onderwerp_boolean__werk_intake,contacten_onderwerp_boolean_beoordelen_taaleis,contacten_onderwerp_boolean_contact_derden,contacten_onderwerp_boolean_contact_met_aanbieder,contacten_onderwerp_boolean_diagnosegesprek,contacten_onderwerp_boolean_documenten__innemen_,contacten_onderwerp_